###### 2020-11-04 수요일

###### 본 실습은.. 데이터의 크기가 커서 코랩을 통해 진행하였습니다.

# 10_앙상블기법_[실습]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

### 데이터 설명
     - Kaggle 경진대회에서 진행했던 `Standard Customer Satisfacion`데이터 셋이다.
     - 이 데이터셋은 최종 Test set의 결과를 제출해야 하기에, Test set에는 target값(Y)이 없어 최종예측결과의 성능을 비교할 수 없다
     - 그러므로 train set을 temp_train set과 temp_test set으로 나누어서 예측을 진행하고 temp_test set에 대하여 예측을 진행한 후 성능을 비교해보자
     

In [ ]:
train = pd.read_csv('/실습데이터/santander-customer-satisfaction/train.csv')
test = pd.read_csv('/실습데이터/santander-customer-satisfaction/test.csv')

In [ ]:
X_train = train.drop(['TARGET'], axis=1, inplace=False)
y_train = train['TARGET']

X_test = test

In [ ]:
X_train.head()

In [ ]:
def model_eval(y_test, y_pred):
    print('Confusion Matrix : \n', confusion_matrix(y_test, y_pred))
    print('accuracy         : ', accuracy_score(y_test, y_pred))
    print('precision        : ', precision_score(y_test, y_pred))
    print('recall           : ', recall_score(y_test, y_pred))
    print('f1               : ', f1_score(y_test, y_pred))
    print('AUC              : ', roc_auc_score(y_test, y_pred))

## 1. 데이터 전처리 없이 XGBoost를 이용한 예측을 시행하라
    - 제공받은 데이터의 Test set에는 Target값이 나타나지 않아 성능을 예측할 수 없다. (실제 데이터나 현업 데이터는 모두 이러한 형식이다.)
    - 그러므로 제공받은 Train set을 임시 Train set과 임시 Test set으로 나누어 성능을 비교해보자
    - 우선은 하이퍼파라미터 튜닝없이 진행한다.

In [ ]:
temp_X_train, temp_X_test, temp_y_train, temp_y_test = train_test_split(X_train, y_train,
                                                                        test_size = 0.2,
                                                                        random_state = 123)

In [ ]:
xgboost = XGBClassifier(random_state=123)
xgboost.fit(temp_X_train, temp_y_train)
temp_y_pred = xgboost.predict(temp_X_test)

In [ ]:
model_eval(temp_y_test, temp_y_pred)

## 이게 정녕.. 결과란 말인가..?

## 2. 성능평가를 early stopping을 통해 조기중단하라

In [ ]:
xgboost = XGBClassifier(random_state=123)

xgboost.fit(temp_X_train, temp_y_train,
            eval_metric                  = 'logloss',
            early_stopping_rounds        = 50,
            eval_set                     = [(temp_X_test, temp_y_test)],
            verbose                      = True)

In [ ]:
temp_y_pred = xgboost.predict(temp_X_test)

In [ ]:
model_eval(temp_y_test, temp_y_pred)

## 위 결과와 다를바가 없다...
## 파라미터 튜닝이 필요해보인다.

## 3. GridSearchCV함수와 Train set을 이용해 하이퍼 파라미터를 튜닝하고 Test set을 예측하라


In [ ]:
xgboost = XGBClassifier(random_state=123)

params = {
    'booster' : ['gbtree'],
    'learning_rate' : [0.1, 0.2],
    'max_depth' : [2, 3],
    'n_estimators' : [200, 300]
}



kfold = StratifiedKFold(n_splits = 3)

grid_xgb = GridSearchCV(xgboost,
                        cv            = kfold,
                        param_grid    = params,
                        scoring       = 'roc_auc',
                        n_jobs        = -1,
                        verbose       = True)

grid_xgb.fit(temp_X_train, temp_y_train)

print('최적의 하이퍼 파라미터는? ', grid_xgb.best_params_)
print('최적의 하이퍼 파라미터 일때 AUC는? ', grid_xgb.best_score_)


# 1. 
# 최적의 하이퍼 파라미터는?  {'booster': 'gbtree', 'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}
# 최적의 하이퍼 파라미터 일때 AUC는?  0.8390561147467664

# 2.
# 최적의 하이퍼 파라미터는?  {'booster': 'gbtree', 'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}
# 최적의 하이퍼 파라미터 일때 AUC는?  0.8390561147467664

# 3.
# 최적의 하이퍼 파라미터는?  {'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}
# 최적의 하이퍼 파라미터 일때 AUC는?  0.8392216606483257

# 4. 
# 최적의 하이퍼 파라미터는?  {'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}
# 최적의 하이퍼 파라미터 일때 AUC는?  0.8392216606483257

In [ ]:
best_model = grid_xgb.best_estimator_
temp_y_pred = best_model.predict(temp_X_test)

In [ ]:
print('최적의 하이퍼파라미터의 예측 결과는?')
model_eval(temp_y_test, temp_y_pred)

## 4. 변수중요도를 시각화하라

In [ ]:
feature_importance = best_model.feature_importances_
top20 = pd.Series(feature_importance, index=X_train.columns).sort_values(ascending=False)[0:20]

sns.barplot(x=top20, y=top20.index)

## 5. Cross Validation을 적용한 Stacking을 적용해 보라

- 데이터의 크기가 크기 때문에, 파라미터 튜닝 없이 DecisionTree와 LogisticRegression을 기본모델로 사용하고, 위에서 찾은 최적의 하이퍼파라미터를 가진 XGBoost를 Stacking의 최종모델로 사용하라.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:

def stacking(model, X_train, y_train, X_test, cv):
    
    k = 0

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)

    kfold = StratifiedKFold(n_splits = cv)
    stacking_X_train = np.zeros((X_train.shape[0], 1))
    stacking_X_test = np.zeros((X_test.shape[0], cv))

    for train_idx, validation_idx in kfold.split(X_train, y_train):
        
        X_train_cv = X_train[train_idx]
        y_train_cv = y_train[train_idx]

        X_validation = X_train[validation_idx]

        model.fit(X_train_cv, y_train_cv)

        stacking_X_train[validation_idx, :] = model.predict(X_validation).reshape(-1,1)
        stacking_X_test[:, k] = model.predict(X_test)

        k += 1
    
    stacking_X_test = stacking_X_test.mean(axis=1).reshape(-1, 1)
    return stacking_X_train, stacking_X_test

In [ ]:
decision = DecisionTreeClassifier(random_state = 123)
logistic = LogisticRegression(random_state = 123)

In [ ]:
decision_train, decision_test = stacking(decision, temp_X_train, temp_y_train, temp_X_test, 3)
logistic_train, logistic_test = stacking(logistic, temp_X_train, temp_y_train, temp_X_test, 3)

In [ ]:
stacking_X_train = np.concatenate([decision_train, logistic_train], axis=1)
stacking_X_test = np.concatenate([decision_test, logistic_test], axis=1)

In [ ]:
best_model.fit(stacking_X_train, temp_y_train)
stacking_y_pred = best_model.predict(stacking_X_test)

In [ ]:
model_eval(temp_y_test, stacking_y_pred)